In [3]:
import pandas as pd

Осуществим предобработку данных с Твиттера, чтобы отчищенный данные в дальнейшем использовать для задачи классификации. Данный датасет содержит негативные (label = 1) и нейтральные (label = 0) высказывания.
Для работы объединим train_df и test_df.

In [4]:
train_df = pd.read_csv('train_tweets.csv')
test_df = pd.read_csv('test_tweets.csv')
combine_df = train_df.append(test_df, ignore_index = True, sort = False)

1) Удалим @user из всех твитов с помощью паттерна "@[\w]*". Для этого создадим функцию: 
 - для того, чтобы найти все вхождения паттерна в тексте, необходимо использовать re.findall(pattern, input_txt)
 - для для замены @user на пробел, необходимо использовать re.sub()

In [5]:
import re

In [6]:
# Delete @user by re
def del_user(x):
    to_del = re.findall("@[\w]*", x)
    for mail in to_del:
        x = re.sub(mail, '', x)
    return x

In [7]:
test_1 = combine_df.tweet[0]
test_1

' @user when a father is dysfunctional and is so selfish he drags his kids into his dysfunction.   #run'

In [8]:
del_user(test_1)

'  when a father is dysfunctional and is so selfish he drags his kids into his dysfunction.   #run'

In [9]:
test_2 = combine_df.tweet[1]
test_2

"@user @user thanks for #lyft credit i can't use cause they don't offer wheelchair vans in pdx.    #disapointed #getthanked"

In [10]:
del_user(test_2)

"  thanks for #lyft credit i can't use cause they don't offer wheelchair vans in pdx.    #disapointed #getthanked"

In [11]:
combine_df.tweet = combine_df.tweet.apply(del_user)

In [12]:
combine_df.tweet[:5]

0      when a father is dysfunctional and is so sel...
1      thanks for #lyft credit i can't use cause th...
2                                  bihday your majesty
3    #model   i love u take with u all the time in ...
4               factsguide: society now    #motivation
Name: tweet, dtype: object

2) Изменим регистр твитов на нижний с помощью .lower().

In [13]:
combine_df.tweet = combine_df.tweet.apply(lambda x: x.lower())

In [14]:
combine_df.tweet[:5]

0      when a father is dysfunctional and is so sel...
1      thanks for #lyft credit i can't use cause th...
2                                  bihday your majesty
3    #model   i love u take with u all the time in ...
4               factsguide: society now    #motivation
Name: tweet, dtype: object

3) Заменим сокращения с апострофами (пример: ain't, can't) на пробел, используя apostrophe_dict. Для этого необходимо сделать функцию: для каждого слова в тексте проверить (for word in text.split()), если слово есть в словаре apostrophe_dict в качестве ключа (сокращенного слова), то заменить ключ на значение (полную версию слова).

In [15]:
def change_word(x:str, to_change:dict):
    # Change apostrophe in text
    origin_list = x.split()
    new_list = []
    for word in origin_list:
        if word in to_change:
            new_list.append(to_change[word])
        else:
            new_list.append(word)
    return ' '.join(new_list)

In [16]:
apostrophe_dict = {
"ain't": "am not / are not",
"aren't": "are not / am not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he had / he would",
"he'd've": "he would have",
"he'll": "he shall / he will",
"he'll've": "he shall have / he will have",
"he's": "he has / he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how has / how is",
"i'd": "I had / I would",
"i'd've": "I would have",
"i'll": "I shall / I will",
"i'll've": "I shall have / I will have",
"i'm": "I am",
"i've": "I have",
"isn't": "is not",
"it'd": "it had / it would",
"it'd've": "it would have",
"it'll": "it shall / it will",
"it'll've": "it shall have / it will have",
"it's": "it has / it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she had / she would",
"she'd've": "she would have",
"she'll": "she shall / she will",
"she'll've": "she shall have / she will have",
"she's": "she has / she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so as / so is",
"that'd": "that would / that had",
"that'd've": "that would have",
"that's": "that has / that is",
"there'd": "there had / there would",
"there'd've": "there would have",
"there's": "there has / there is",
"they'd": "they had / they would",
"they'd've": "they would have",
"they'll": "they shall / they will",
"they'll've": "they shall have / they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we had / we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what shall / what will",
"what'll've": "what shall have / what will have",
"what're": "what are",
"what's": "what has / what is",
"what've": "what have",
"when's": "when has / when is",
"when've": "when have",
"where'd": "where did",
"where's": "where has / where is",
"where've": "where have",
"who'll": "who shall / who will",
"who'll've": "who shall have / who will have",
"who's": "who has / who is",
"who've": "who have",
"why's": "why has / why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you had / you would",
"you'd've": "you would have",
"you'll": "you shall / you will",
"you'll've": "you shall have / you will have",
"you're": "you are",
"you've": "you have"
}

In [17]:
test_1 = combine_df.tweet[1]
test_1

"  thanks for #lyft credit i can't use cause they don't offer wheelchair vans in pdx.    #disapointed #getthanked"

In [18]:
change_word(test_1, apostrophe_dict)

'thanks for #lyft credit i cannot use cause they do not offer wheelchair vans in pdx. #disapointed #getthanked'

In [19]:
combine_df.tweet = combine_df.tweet.apply(lambda x: change_word(x, apostrophe_dict))

4) Заменим сокращения на их полные формы, используя short_word_dict. Для этого воспользуемся функцией, используемой в предыдущем пункте.

In [20]:
short_word_dict = {
"121": "one to one",
"a/s/l": "age, sex, location",
"adn": "any day now",
"afaik": "as far as I know",
"afk": "away from keyboard",
"aight": "alright",
"alol": "actually laughing out loud",
"b4": "before",
"b4n": "bye for now",
"bak": "back at the keyboard",
"bf": "boyfriend",
"bff": "best friends forever",
"bfn": "bye for now",
"bg": "big grin",
"bta": "but then again",
"btw": "by the way",
"cid": "crying in disgrace",
"cnp": "continued in my next post",
"cp": "chat post",
"cu": "see you",
"cul": "see you later",
"cul8r": "see you later",
"cya": "bye",
"cyo": "see you online",
"dbau": "doing business as usual",
"fud": "fear, uncertainty, and doubt",
"fwiw": "for what it's worth",
"fyi": "for your information",
"g": "grin",
"g2g": "got to go",
"ga": "go ahead",
"gal": "get a life",
"gf": "girlfriend",
"gfn": "gone for now",
"gmbo": "giggling my butt off",
"gmta": "great minds think alike",
"h8": "hate",
"hagn": "have a good night",
"hdop": "help delete online predators",
"hhis": "hanging head in shame",
"iac": "in any case",
"ianal": "I am not a lawyer",
"ic": "I see",
"idk": "I don't know",
"imao": "in my arrogant opinion",
"imnsho": "in my not so humble opinion",
"imo": "in my opinion",
"iow": "in other words",
"ipn": "I’m posting naked",
"irl": "in real life",
"jk": "just kidding",
"l8r": "later",
"ld": "later, dude",
"ldr": "long distance relationship",
"llta": "lots and lots of thunderous applause",
"lmao": "laugh my ass off",
"lmirl": "let's meet in real life",
"lol": "laugh out loud",
"ltr": "longterm relationship",
"lulab": "love you like a brother",
"lulas": "love you like a sister",
"luv": "love",
"m/f": "male or female",
"m8": "mate",
"milf": "mother I would like to fuck",
"oll": "online love",
"omg": "oh my god",
"otoh": "on the other hand",
"pir": "parent in room",
"ppl": "people",
"r": "are",
"rofl": "roll on the floor laughing",
"rpg": "role playing games",
"ru": "are you",
"shid": "slaps head in disgust",
"somy": "sick of me yet",
"sot": "short of time",
"thanx": "thanks",
"thx": "thanks",
"ttyl": "talk to you later",
"u": "you",
"ur": "you are",
"uw": "you’re welcome",
"wb": "welcome back",
"wfm": "works for me",
"wibni": "wouldn't it be nice if",
"wtf": "what the fuck",
"wtg": "way to go",
"wtgp": "want to go private",
"ym": "young man",
"gr8": "great"
}



In [21]:
combine_df.tweet = combine_df.tweet.apply(lambda x: change_word(x, short_word_dict))

5) Заменим эмотиконы (пример: ":)" = "happy") на пробелы, используя emoticon_dict. Для этого воспользуемся функцией, используемой в предыдущем пункте.

In [22]:
emoticon_dict = {
":)": "happy",
":‑)": "happy",
":-]": "happy",
":-3": "happy",
":->": "happy",
"8-)": "happy",
":-}": "happy",
":o)": "happy",
":c)": "happy",
":^)": "happy",
"=]": "happy",
"=)": "happy",
"<3": "happy",
":-(": "sad",
":(": "sad",
":c": "sad",
":<": "sad",
":[": "sad",
">:[": "sad",
":{": "sad",
">:(": "sad",
":-c": "sad",
":-< ": "sad",
":-[": "sad",
":-||": "sad"
}

In [23]:
combine_df.tweet = combine_df.tweet.apply(lambda x: change_word(x, emoticon_dict))

6) Заменим пунктуацию на пробелы, используя re.sub() и паттерн r'[^\w\s]'.

In [24]:
import string

remove = string.punctuation
pattern = r"[{}]".format(remove) 

In [25]:
# change to space
def to_space(pattern_:str, x:str, to_:str=' '):
    # to_space = re.findall(pattern, x)
    x = re.sub(pattern, to_, x)
    return x

In [26]:
test_2 = combine_df.tweet[1]
test_2

'thanks for #lyft credit i cannot use cause they do not offer wheelchair vans in pdx. #disapointed #getthanked'

In [27]:
to_space(pattern_=pattern, x=test_2, to_=' ')

'thanks for  lyft credit i cannot use cause they do not offer wheelchair vans in pdx   disapointed  getthanked'

In [28]:
combine_df.tweet = combine_df.tweet.apply(lambda x: to_space(pattern_=pattern, x=x, to_=' '))

In [29]:
# 1? Почему когда делаешь цикл как в первом задании, в результате получаешь точки?

7) Заменим спец. символы на пробелы, используя re.sub() и паттерн r'[^a-zA-Z0-9]'.

In [30]:
# 2? Почему петтер для слов и цифр а надо спец. символы заменить?
# 3? По идее решено выше?

8) Заменим числа на пробелы, используя re.sub() и паттерн r'[^a-zA-Z]'.

In [31]:
test_1 = combine_df.tweet[5]
test_1

' 2 2  huge fan fare and big talking before they leave  chaos and pay disputes when they get there   allshowandnogo'

In [32]:
to_space(pattern_= r"(\b|\s+\-?|^\-?)(\d+|\d*\.\d+)\b", x=test_1, to_=' ')

' 2 2  huge fan fare and big talking before they leave  chaos and pay disputes when they get there   allshowandnogo'

In [33]:
re.findall(r"(\b|\s+\-?|^\-?)(\d+|\d*\.\d+)\b", test_1)

[(' ', '2'), (' ', '2')]

In [34]:
re.findall(r"\d", test_1)

['2', '2']

In [35]:
to_space(pattern_= r"\d", x=test_1, to_=' ')

' 2 2  huge fan fare and big talking before they leave  chaos and pay disputes when they get there   allshowandnogo'

In [36]:
re.sub('\d', '', test_1)

'    huge fan fare and big talking before they leave  chaos and pay disputes when they get there   allshowandnogo'

In [ ]:
# 4?  Почему код функции to_space отрабатывает не корректно ? 

9. Удалим из текста слова длиной в 1 символ, используя ' '.join([w for w in x.split() if len(w)>1]).

In [37]:
combine_df.tweet = combine_df.tweet.apply(lambda x: ' '.join([w for w in x.split() if len(w)>1]))


10) Поделим твиты на токены с помощью nltk.tokenize.word_tokenize, создав новый столбец 'tweet_token'.

In [41]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [42]:
combine_df['tweet_token'] = 0
combine_df.tweet_token = combine_df.tweet.apply(lambda x: nltk.tokenize.word_tokenize(x))

In [43]:
# 5? почему меня вежливо попросили подгрузить nltk.download('punkt')?

12) Применим стемминг к токенам с помощью nltk.stem.PorterStemmer. Создадим столбец 'tweet_stemmed' после применения стемминга.

In [60]:
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()
combine_df['tweet_stemmed'] = 0
combine_df.tweet_stemmed = combine_df.tweet_token.apply(lambda x: [stemmer.stem(i) for i in x])

13) Применим лемматизацию к токенам с помощью nltk.stem.wordnet.WordNetLemmatizer. Создадим столбец 'tweet_lemmatized' после применения лемматизации.

In [63]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()
combine_df['tweet_lemmatized'] = 0
combine_df.tweet_lemmatized = combine_df.tweet_token.apply(lambda x: [lemmatizer.lemmatize(i, wordnet.VERB) for i in x])

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


14) Сохраним результат предобработки в pickle-файл.


In [64]:
combine_df.to_pickle("dummy.pkl")